# Istari AIAA Hands on Keyboard Event

## About this doc

This notebook demonstrates the Istari platform workflow for a Group 3 expendable UAV (tailless flying wing).
You will modify wing design parameters, run an nTop structural analysis, and verify compliance with system requirements.

**Goal**: Achieve the lowest Structure Weight while meeting all requirements.

In [ ]:
#@title User Data { display-mode: "form" }
!pip install istari-digital-client -q

from google.colab import userdata

#@markdown Your `ISTARI_PAT` should be stored in Colab Secrets (🔑 in sidebar)

ISTARI_ENVIRONMENT_URL = "https://fileservice-v2.stage.istari.app"
ISTARI_PAT = userdata.get('ISTARI_PAT')

#@markdown ---

NTOP_MODEL_ID = "22c2c7cc-aa0f-4492-8caf-4f08bd6d5f3d" #@param {type:"string"}

print(f"✓ Ready" if ISTARI_PAT else "✗ PAT not found (check Colab Secrets)")

In [ ]:
#@title Wing Design Parameters { display-mode: "form" }

Length_Overall_in = 99.9 #@param {type:"number"}
Wingspan_in = 144.0 #@param {type:"number"}
Leading_Edge_Sweep_Inboard_deg = 46.5 #@param {type:"number"}
Leading_Edge_Sweep_Outboard_deg = 46.5 #@param {type:"number"}
Trailing_Edge_Sweep_Inboard_deg = -46.5 #@param {type:"number"}
Trailing_Edge_Sweep_Outboard_deg = 15.0 #@param {type:"number"}
Panel_Break_Span_Fraction = 0.3 #@param {type:"number"}

# Map to internal variable names
loa_in = Length_Overall_in
span = Wingspan_in
le_sweep_p1 = Leading_Edge_Sweep_Inboard_deg
le_sweep_p2 = Leading_Edge_Sweep_Outboard_deg
te_sweep_p1 = Trailing_Edge_Sweep_Inboard_deg
te_sweep_p2 = Trailing_Edge_Sweep_Outboard_deg
panel_break_span_pct = Panel_Break_Span_Fraction

In [ ]:
#@title Analyze and Verify { display-mode: "form" }
import json
import time
import logging
from istari_digital_client.client import Client
from istari_digital_client.configuration import Configuration
from istari_digital_client import JobStatusName

# === Material Cost Estimates ($/lb) ===
COMPOSITE_COST_PER_LB = 85.00  # Carbon fiber composite
METAL_COST_PER_LB = 12.50      # Aluminum alloy

# === Connect to Istari ===
print("Connecting to Istari...")
client = Client(
    config=Configuration(
        registry_url=ISTARI_ENVIRONMENT_URL,
        registry_auth_token=ISTARI_PAT,
    )
)
ntop_model = client.get_model(NTOP_MODEL_ID)
print(f"✓ Connected ({ntop_model.display_name or ntop_model.name})")

# === Step 1: Build input parameters ===
print(f"Configuring wing: LOA={loa_in}in, Span={span}in, LE Sweep={le_sweep_p1}°/{le_sweep_p2}°, TE Sweep={te_sweep_p1}°/{te_sweep_p2}°")

input_json_data = {
    "inputs": [
        {"name": "LOA In", "type": "real", "units": "in", "value": loa_in},
        {"name": "Span", "type": "real", "units": "in", "value": span},
        {"name": "LE Sweep P1", "type": "real", "units": "deg", "value": le_sweep_p1},
        {"name": "LE Sweep P2", "type": "real", "units": "deg", "value": le_sweep_p2},
        {"name": "TE Sweep P1", "type": "real", "units": "deg", "value": te_sweep_p1},
        {"name": "TE Sweep P2", "type": "real", "units": "deg", "value": te_sweep_p2},
        {"name": "Panel Break Span %", "type": "real", "value": panel_break_span_pct},
        {"name": "MAIN PATH", "type": "file_path", "value": "/home/bradrothenberg/nTopGrp3/output/"}
    ]
}

# === Step 2: Run nTop analysis ===
print("Starting nTop analysis...")
logging.getLogger('istari_digital_client').setLevel(logging.CRITICAL)

run_job = client.add_job(
    model_id=ntop_model.id,
    function="@ntop:run_model",
    tool_name="ntopcl",
    tool_version="5.30",
    operating_system="RHEL 8",
    parameters={"ntop_input_json": input_json_data},
)

last_status = None
while run_job.status.name not in [JobStatusName.COMPLETED, JobStatusName.FAILED]:
    time.sleep(5)
    run_job = client.get_job(run_job.id)
    if run_job.status.name != last_status:
        status_msg = str(run_job.status.name).replace("_", " ").lower()
        print(f"nTop analysis is {status_msg}")
        last_status = run_job.status.name

logging.getLogger('istari_digital_client').setLevel(logging.WARNING)

if run_job.status.name == JobStatusName.FAILED:
    print("❌ There was an error in the nTop analysis")
    raise Exception("nTop analysis failed")

print("✓ nTop analysis complete")

# === Step 3: Extract outputs ===
print("Extracting results...")

ntop_model = client.get_model(ntop_model.id)

output_values = {}
for artifact in ntop_model.artifacts:
    if artifact.name == "output.json":
        output_data = json.loads(artifact.read_text())
        if isinstance(output_data, list):
            for item in output_data:
                if isinstance(item, dict) and item.get("type") == "json":
                    output_values = item.get("value", {}).get("jsonObject", {})

# Extract all available outputs
weight_composite = output_values.get("Weight_Composite (lbm)")
weight_metal = output_values.get("Weight_Metal (lbm)")
wingtip_displacement = output_values.get("wingtipDisplacement (in)")
volume = output_values.get("Volume (in^3)") or output_values.get("Volume")
surface_area = output_values.get("Surface_Area (in^2)") or output_values.get("Surface_Area")

# Calculate derived values
total_weight = (weight_composite or 0) + (weight_metal or 0)
composite_cost = (weight_composite or 0) * COMPOSITE_COST_PER_LB
metal_cost = (weight_metal or 0) * METAL_COST_PER_LB
total_material_cost = composite_cost + metal_cost

# === Step 4: Verify requirements ===
requirements = {
    "Total_Weight": {"max": 500.0, "units": "lb"},
    "Wingtip_Displacement": {"max": 0.5, "units": "in"}
}

weight_ok = total_weight > 0 and total_weight <= requirements["Total_Weight"]["max"]
displacement_ok = wingtip_displacement is not None and wingtip_displacement <= requirements["Wingtip_Displacement"]["max"]
all_pass = weight_ok and displacement_ok

# === Display Results ===
print("\n" + "=" * 55)
print("               ANALYSIS COMPLETE")
print("=" * 55)

print("\n  STRUCTURAL PROPERTIES")
print("  " + "-" * 40)
print(f"    Composite Weight:    {weight_composite:>10.2f} lb")
print(f"    Metal Weight:        {weight_metal:>10.2f} lb")
print(f"    Total Weight:        {total_weight:>10.2f} lb  {'✅' if weight_ok else '❌'}")
print(f"    Wingtip Deflection:  {wingtip_displacement:>10.4f} in  {'✅' if displacement_ok else '❌'}")

if volume is not None:
    print(f"    Volume:              {volume:>10.2f} in³")
if surface_area is not None:
    print(f"    Surface Area:        {surface_area:>10.2f} in²")

print("\n  MATERIAL COST ESTIMATE")
print("  " + "-" * 40)
print(f"    Composite (${COMPOSITE_COST_PER_LB:.0f}/lb): ${composite_cost:>10,.2f}")
print(f"    Metal (${METAL_COST_PER_LB:.0f}/lb):      ${metal_cost:>10,.2f}")
print(f"    Total Material Cost:   ${total_material_cost:>10,.2f}")

print("\n" + "=" * 55)
if all_pass:
    print("  ✅ ALL REQUIREMENTS PASSED!")
    print(f"  🎯 Structure Weight: {total_weight:.2f} lb | Cost: ${total_material_cost:,.2f}")
    print("     Can you optimize further?")
else:
    print("  ❌ REQUIREMENTS NOT MET")
    if not weight_ok:
        print(f"     - Weight {total_weight:.1f} lb exceeds {requirements['Total_Weight']['max']} lb max")
    if not displacement_ok:
        print(f"     - Deflection {wingtip_displacement:.4f} in exceeds {requirements['Wingtip_Displacement']['max']} in max")
print("=" * 55)